In [ ]:
import yaml
from pathlib import Path
from jinja2 import Template
import os
import toml
from google.genai import types
from google import genai 
import time

In [ ]:
class PromptLoader:
    def __init__(self, path: str = "prompts.yaml"):
        with open(path, "r", encoding="utf-8") as f:
            self.prompts = yaml.safe_load(f)

    def render(self, name: str, **kwargs) -> str:
        """Render a named prompt with given variables."""
        template = Template(self.prompts[name])
        return template.render(**kwargs)

In [ ]:
input_path = "data/bfp-a3447q_chunked.txt"
output_path = "data/bfp-a3447q_context.txt"

In [ ]:
import pathlib, json
json_read = pathlib.Path(input_path).read_text()
data = json.loads(json_read)

In [ ]:
paragraph_dict = dict()
context_dict = dict()
loader = PromptLoader("data/prompts.yaml")

In [ ]:
config = toml.load("../../.streamlit/secrets.toml")
os.environ["GEMINI_API_KEY"] = config["gemini"]["GOOGLE_API_KEY"]

In [ ]:
client = genai.Client()
print("Client initialized")

In [ ]:
root_chapter_ix = 0
window_size = 3 # max number of chunks to be taken before and ahead of selected chunk
for index, chunk in enumerate(data):
    if index in context_dict.keys():
        continue
    if chunk[0] == 1:
        root_chapter_ix = index
    if 2*len(chunk[1]) > len(chunk[-1]): # if chapter is empty, skip its context creation
        print(f'{index}) Paragraphs not generated for chapter: {chunk[1]}')
        continue
    paragraph_start = max(root_chapter_ix, index - window_size)
    paragraph_end = min(index + window_size + 1, len(data))
    paragraph_list = [ d[-1] for d in data[paragraph_start:paragraph_end]]
    paragraphs = '\n'.join(paragraph_list)
    print(f'{index}) Paragraphs generated in range [{paragraph_start}:{paragraph_end-1}] for chapter: {chunk[1]}')
    paragraph_dict[index] = paragraphs
    prompt = loader.render(
        "context_extension",
        chunk=chunk[-1],
        neighbors=paragraphs
    )
    print("\rPrompting standby...",end='')
    time.sleep(2)
    print("\rWaiting for generation...",end='')
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0)
        ),
        contents=prompt
    )
    print(f'\r\tContext generated with end: {response.text[-50:]}')
    context_dict[index] = response.text

In [ ]:
import json
json_file=json.dumps(context_dict, indent=2)
# Exporting data to output file for storage
with open(output_path, mode='w+') as f_out:
    f_out.write(json_file)